# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
# M WALTON - CONVERT CITY TO TITLE CASE
city_data_df["City"] = city_data_df["City"].str.title()
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Camana,-16.6228,-72.7111,16.92,83,79,3.16,PE,1688771832
1,1,Wailua Homesteads,22.0669,-159.3780,29.44,70,20,7.72,US,1688771833
2,2,Adamstown,-25.0660,-130.1015,21.70,68,18,6.57,PN,1688771703
3,3,West Island,-12.1568,96.8225,25.99,73,40,6.69,CC,1688771833
4,4,Kenora,49.8167,-94.4337,20.43,40,61,2.68,CA,1688771833


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [38]:
%%capture --no-display

# Configure the map plot
city_data_df["Lat"]
city_data_df["Lng"]
city_data_df["Humidity"]
Humidity_map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    size = "Humidity",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    color = "City"
    
)

#points.opts(color ="z",legend_position = "left")

# Display the map
Humidity_map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [40]:
# Narrow down cities that fit criteria and drop any results with null values

ideal_temp = (city_data_df["Max Temp"] >= 21) & (city_data_df["Max Temp"] <= 27) & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] ==0)
ideal_temp_df = city_data_df[ideal_temp]


# Drop any rows with null values - Code Ref W3 schools, data unknown
clean_ideal_temp_df = ideal_temp_df.dropna()

# Display sample data
clean_ideal_temp_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
78,78,Isla Vista,34.4133,-119.8610,26.30,73,0,3.60,US,1688771853
110,110,Fort Mcmurray,56.7268,-111.3810,24.77,35,0,2.06,CA,1688771863
113,113,Jamestown,42.0970,-79.2353,23.34,58,0,3.60,US,1688771863
118,118,Aqsay,51.1678,52.9950,22.49,53,0,3.96,KZ,1688771865
122,122,Island Harbour,50.7240,-1.2758,22.21,71,0,3.13,GB,1688771866
180,180,Naxos,37.1056,25.3764,26.84,76,0,0.45,GR,1688771886
221,221,Mizdah,31.4451,12.9801,25.53,34,0,3.55,LY,1688771897
256,256,Ad Darbasiyah,37.0728,40.6520,26.72,53,0,3.58,SY,1688771908
322,322,Casablanca,33.5928,-7.6192,24.07,60,0,2.68,MA,1688771925
410,410,Vila Velha,-20.3297,-40.2925,21.97,83,0,3.60,BR,1688771948


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity 
#Code Ref:  Zach, 2022
hotel_df = clean_ideal_temp_df[["City", "Country","Lat", "Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
78,Isla Vista,US,34.4133,-119.8610,73,
110,Fort Mcmurray,CA,56.7268,-111.3810,35,
113,Jamestown,US,42.0970,-79.2353,58,
118,Aqsay,KZ,51.1678,52.9950,53,
122,Island Harbour,GB,50.7240,-1.2758,71,
180,Naxos,GR,37.1056,25.3764,76,
221,Mizdah,LY,31.4451,12.9801,34,
256,Ad Darbasiyah,SY,37.0728,40.6520,53,
322,Casablanca,MA,33.5928,-7.6192,60,
410,Vila Velha,BR,-20.3297,-40.2925,83,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
limit = 10
radius = 10000
params = {
    "categories": "accommodation.hotel", 
    "apiKey":geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    city = row["City"]
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    params["text"] = f"{city}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results 
    # format to title case
    print(f"{hotel_df.loc[index, 'City'].title()} - nearest hotel: {hotel_df.loc[index, 'Hotel Name'].title()}")

# Display sample data
hotel_df

Starting hotel search
Isla Vista - nearest hotel: The Club & Guest House
Fort Mcmurray - nearest hotel: Nomad Hotel And Suites
Jamestown - nearest hotel: Doubletree Jamestown
Aqsay - nearest hotel: No Hotel Found
Island Harbour - nearest hotel: Premier Inn
Naxos - nearest hotel: Chateau Zevgoli
Mizdah - nearest hotel: No Hotel Found
Ad Darbasiyah - nearest hotel: No Hotel Found
Casablanca - nearest hotel: Hotel De Prince De Paris
Vila Velha - nearest hotel: Hotel Prainha
Vilhena - nearest hotel: No Hotel Found
Menomonie - nearest hotel: Cobblestone Inn


,City,Country,Lat,Lng,Humidity,Hotel Name
78,Isla Vista,US,34.4133,-119.8610,73,The Club & Guest House
110,Fort Mcmurray,CA,56.7268,-111.3810,35,Nomad Hotel and Suites
113,Jamestown,US,42.0970,-79.2353,58,DoubleTree Jamestown
118,Aqsay,KZ,51.1678,52.9950,53,No hotel found
122,Island Harbour,GB,50.7240,-1.2758,71,Premier Inn
180,Naxos,GR,37.1056,25.3764,76,Chateau Zevgoli
221,Mizdah,LY,31.4451,12.9801,34,No hotel found
256,Ad Darbasiyah,SY,37.0728,40.6520,53,No hotel found
322,Casablanca,MA,33.5928,-7.6192,60,Hotel de Prince de Paris
410,Vila Velha,BR,-20.3297,-40.2925,83,Hotel Prainha


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the map plot
hotel_df["Lat"]
hotel_df["Lng"]
hotel_df["City"]
hotel_df["Hotel Name"]
hotel_map_plot_1 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    hover_cols=["City", "Humidity","Hotel Name","Country"],
    geo = True,
    tiles = "ESRI",
    size = 100,
    frame_width = 800,
    frame_height = 600,
    color = "yellow"
)

# Display the map
hotel_map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)